# GPT Data Collection

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup

import pprintpp
pp = pprintpp.PrettyPrinter(indent=4)

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [55]:
members_gpt = pd.read_pickle("../Data-Acquisition/members_gpt.pkl")
members_gpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4425 entries, 0 to 4424
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   member                 4425 non-null   object
 1   said_something_racist  4425 non-null   object
 2   went_private_school    4425 non-null   object
 3   said_something_sexist  4425 non-null   object
 4   is_part_of_lgbtq       4425 non-null   object
 5   ethnicity              4425 non-null   object
 6   is_from                4425 non-null   object
dtypes: object(7)
memory usage: 242.1+ KB


# Adding a members id column

In [56]:
members = pd.read_pickle("../Data-Acquisition/members.pkl")
members_gpt = members_gpt.rename(columns={"member": "full_title"})
members = members[["id", "full_title"]]
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763 entries, 0 to 4762
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4763 non-null   int64 
 1   full_title  4763 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.6+ KB


In [60]:
inner_merged = pd.merge(members_gpt, members, on="full_title").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4670 entries, 0 to 4669
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   full_title             4670 non-null   object
 1   said_something_racist  4670 non-null   object
 2   went_private_school    4670 non-null   object
 3   said_something_sexist  4670 non-null   object
 4   is_part_of_lgbtq       4670 non-null   object
 5   ethnicity              4670 non-null   object
 6   is_from                4670 non-null   object
 7   id                     4670 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 328.4+ KB


In [ ]:
test